In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch_geometric as pyg
import torch_geometric.datasets as pygd
import warnings
from sklearn.exceptions import UndefinedMetricWarning

warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

In [24]:
from gpn.experiments.multiple_run_experiment import MultipleRunExperiment
from gpn.experiments.transductive_experiment import TransductiveExperiment
from gpn.utils.config import (
    DataConfiguration,
    ModelConfiguration,
    RunConfiguration,
    TrainingConfiguration,
)


def create_experiment(model_name):
    run_cfg = RunConfiguration(
        job="train",
        eval_mode="default",
        experiment_directory="./.cache",
        save_model=True,
        gpu=0,
        experiment_name="ood_loc",
        num_inits=1,
        num_splits=1,
    )
    data_cfg = DataConfiguration(
        dataset="AmazonComputers",
        split_no=1,
        root="./data",
        ood_flag=True,
        train_samples_per_class=0.05,
        val_samples_per_class=0.15,
        test_samples_per_class=0.8,
        split="random",
        ood_setting="poisoning",
        ood_type="leave_out_classes",
        ood_num_left_out_classes=-1,
        ood_leave_out_last_classes=True,
    )

    model_cfg = ModelConfiguration(
        model_name=model_name,
        seed=42,
        init_no=1,
        dim_hidden=64,
        dropout_prob=0.5,
        K=10,
        add_self_loops=True,
        maf_layers=0,
        gaussian_layers=0,
        use_batched_flow=True,
        loss_reduction="sum",
        approximate_reg=True,
        flow_weight_decay=0.0,
        pre_train_mode="flow",
        alpha_evidence_scale="latent-new",
        alpha_teleport=0.1,
        entropy_reg=0.0001,
        dim_latent=16,
        radial_layers=10,
        sparse_propagation=True,
        sparse_x_prune_threshold=0.01
    )

    train_cfg = TrainingConfiguration(
        epochs=100000,
        stopping_mode="default",
        stopping_patience=50,
        stopping_restore_best=True,
        stopping_metric="val_CE",
        stopping_minimize=True,
        finetune_epochs=0,
        warmup_epochs=5,
        lr=0.01,
        weight_decay=0.001,
    )

    return MultipleRunExperiment(run_cfg, data_cfg, model_cfg, train_cfg)


# gpn_e = create_experiment("GPN")
lop_e = create_experiment("GPN_LOP")
res_lop = lop_e.run()
# res = gpn_e.run()
res_lop

Epoch 1/5:
 [Elapsed 0:00:00 | 1.49 it/s] REG: 0.65403, UCE: 1151.37036, train_CE: 1.76393, train_ECE: 0.43378, train_accuracy: 0.22199, train_average_entropy: 1.06049, train_avg_prediction_confidence_aleatoric: 0.65577, train_avg_prediction_confidence_epistemic: 2245.25562, train_avg_sample_confidence_aleatoric: 0.65577, train_avg_sample_confidence_epistemic: 3423.84106, train_avg_sample_confidence_epistemic_entropy: 13.89416, train_avg_sample_confidence_features: 3424.21606, train_avg_sample_confidence_neighborhood: nan, train_brier_score: 0.94648, train_confidence_aleatoric_apr: 0.22179, train_confidence_aleatoric_auroc: 0.47620, train_confidence_epistemic_apr: 0.26828, train_confidence_epistemic_auroc: 0.58216, train_confidence_structure_apr: nan, train_confidence_structure_auroc: nan, val_CE: 1.76763, val_ECE: 0.43476, val_accuracy: 0.22101, val_average_entropy: 1.06050, val_avg_prediction_confidence_aleatoric: 0.65577, val_avg_prediction_confidence_epistemic: 2245.18115, val_avg_

KeyboardInterrupt: 

In [ ]:
import pandas as pd
from gpn.utils.utils import results_dict_to_df

df_lop = results_dict_to_df(res_lop)
df = results_dict_to_df(res)

pd.set_option("display.max_rows", None)
(pd.DataFrame(
    dict(
        val_gpn=df["val"],
        test_gpn=df["test"],
        val_lop=df_lop["val"],
        test_lop=df_lop["test"],
    )
)).round(4).dropna()

In [ ]:
from gpn.data.dataset_manager import DatasetManager
from gpn.data.dataset_provider import InMemoryDatasetProvider
import torch_sparse as ts

ds = InMemoryDatasetProvider(
    DatasetManager(
        dataset="AmazonPhotos",
        split_no=1,
        root="./data",
        ood_flag=True,
        train_samples_per_class=0.05,
        val_samples_per_class=0.15,
        test_samples_per_class=0.8,
        split="random",
        ood_setting="poisoning",
        ood_type="leave_out_classes",
        ood_num_left_out_classes=-1,
        ood_leave_out_last_classes=True,
    )
).to_sparse()

batch = list(ds.loader())[0]

batch.adj_t.dtype()

In [ ]:
N = batch.x.shape[0]
adj_t = batch.adj_t.cuda()
adj_t

In [52]:
import torch_sparse as ts

adj_t = ts.SparseTensor.from_dense(
    torch.Tensor([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
).cuda()
x = torch.Tensor([1, 10, 100]).view(-1, 1).cuda()
N = x.shape[0]
adj_t /= x
print(adj_t.dtype())
adj_t.set_diag(torch.tensor([6,2,3], dtype=torch.float32).cuda()).to_dense()

torch.float32


tensor([[6.0000, 1.0000, 0.0000],
        [0.1000, 2.0000, 0.1000],
        [0.0000, 0.0100, 3.0000]], device='cuda:0')

In [28]:
from gpn.layers.appnp_propagation import APPNPPropagation

prop = APPNPPropagation(1, 0, 0, normalization=None)
# x = torch.diag(torch.ones(N, device="cuda"))
x = ts.SparseTensor.eye(N, dtype=torch.float32, device="cuda")  # type: ignore

p: ts.SparseTensor = prop(x, adj_t)

p.size(0)

SparseTensor(row=tensor([0, 1, 1, 2], device='cuda:0'),
             col=tensor([1, 0, 2, 1], device='cuda:0'),
             val=tensor([1., 1., 1., 1.], device='cuda:0'),
             size=(3, 3), nnz=4, density=44.44%) torch.Size([7])


3

In [ ]:
ts.SparseTensor.from_edge_index(torch.tensor([[0,1],[0,1]]),sparse_sizes=(10,3)).dtype()

In [43]:
ts.SparseTensor.from_.get_diag()

TypeError: <lambda>() missing 1 required positional argument: 'self'